<a href="https://colab.research.google.com/github/jakejmedeiros/btc-prediction/blob/main/btc_ai_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import requests
from sklearn.preprocessing import MinMaxScaler
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
import matplotlib.pyplot as plt

# fetches data from coindesk API
inital_data = requests.get('https://api.coindesk.com/v1/bpi/historical/close.json?start=2012-01-01&end=2021-01-01').json()
price_data = np.array(list(inital_data['bpi'].values()))

# scales data between 0,1
scale = MinMaxScaler(feature_range=(0,1))
scaled_data = scale.fit_transform(price_data.reshape(-1,1))

data_train = []
predict_train = []

training_days = 200

# stores data in the arrays
for i in range(training_days, len(scaled_data)):
  # data training array contains data excluding the last training_days (200)
  data_train.append(scaled_data[i - training_days: i, 0])
  # prediction data contains all data in range
  predict_train.append(scaled_data[i, 0])

# transfrom training data into numpy arrays
data_train, predict_train = np.array(data_train), np.array(predict_train)

# reshapes the data into a 3-D array
data_train = np.reshape(data_train, (data_train.shape[0], data_train.shape[1], 1))

# creating the neural network
network = Sequential()

# adding the LSTM layers 
network.add(LSTM(units = 80, recurrent_dropout = 0.4, return_sequences = True, input_shape = (data_train.shape[1], 1)))
network.add(LSTM(units = 70, recurrent_dropout = 0.2, return_sequences = True))
network.add(LSTM(units = 65, recurrent_dropout = 0.2))
# dense output layer with 1 unit
network.add(Dense(units = 1))

# trains the network based on above data arrays
network.compile(optimizer = 'adam', loss = 'mean_squared_error')
network.fit(data_train, predict_train, epochs = 40, batch_size = 32)


In [ ]:
# testing the neural network

# get data for the next month
test_data = requests.get('https://api.coindesk.com/v1/bpi/historical/close.json?start=2021-01-01&end=2021-02-01').json()
test_prices = np.array(list(test_data['bpi'].values()))

# array of all data
all_data = tf.concat((price_data, test_prices), axis = 0)

# creates the input for the neural network 
network_input = (all_data[len(all_data) - len(test_prices) - training_days:])
network_input = scale.transform(np.array(network_input).reshape(-1, 1))

# predictions
test_arr = []

for i in range(training_days, len(network_input)):
  test_arr.append(network_input[i - training_days: i, 0])

test_arr = np.array(test_arr)
test_arr = np.reshape(test_arr, (test_arr.shape[0], test_arr.shape[1], 1))

predicted_price = network.predict(test_arr)
predicted_price = scale.inverse_transform(predicted_price)

# plot predictions for accuracy visualization
plt.plot(test_prices, color = "green", label = "Actual BTC Price")
plt.plot(predicted_price, color = "red", label = "Predicted BTC Price")
plt.title("Bitcoin Price Prediction")
plt.xlabel("Time")
plt.ylabel("BTC Price")
plt.legend()
plt.show()


